In [8]:
import cv2
import numpy as np
import torch
from torchvision import transforms
from PIL import Image, ImageOps

In [9]:
def load_model():
    # Load the DeepLabv3 model
    model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
    model.eval()
    return model

In [10]:
class_name_to_id = {
    "background": 0,
    "aeroplane": 1,
    "bicycle": 2,
    "bird": 3,
    "boat": 4,
    "bottle": 5,
    "bus": 6,
    "car": 7,
    "cat": 8,
    "chair": 9,
    "cow": 10,
    "diningtable": 11,
    "dog": 12,
    "horse": 13,
    "motorbike": 14,
    "person": 15,
    "pottedplant": 16,
    "sheep": 17,
    "sofa": 18,
    "train": 19,
    "tvmonitor": 20
}

In [11]:
def inpaint_image(model, input_file, target_classes):
    # Open and convert the image to RGB
    input_image = Image.open(input_file).convert("RGB")
    
    # Convert the PIL Image to a NumPy array since OpenCV works with NumPy arrays
    cv_image = np.array(input_image)
    # Convert RGB to BGR for OpenCV
    cv_image = cv_image[:, :, ::-1].copy() 

    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)  # Create a mini-batch as expected by the model

    # Move the input and model to GPU for speed if available
    if torch.cuda.is_available():
        input_batch = input_batch.to('cuda')
        model.to('cuda')

    with torch.no_grad():
        output = model(input_batch)['out'][0]
    output_predictions = output.argmax(0)

    # Create a binary mask of the objects to remove
    mask = np.isin(output_predictions.cpu().numpy(), target_classes).astype(np.uint8)
    # Objects marked for removal are True, so multiply by 255 to make the mask suitable for OpenCV
    mask = mask * 255

    # Inpaint the image using the mask
    # Note: INPAINT_TELEA is one inpainting method offered by OpenCV. You might experiment with INPAINT_NS for different results.
    inpainted_image_bgr = cv2.inpaint(cv_image, mask, inpaintRadius=3, flags=cv2.INPAINT_NS)

    # Convert the inpainted image back to RGB
    inpainted_image_rgb = cv2.cvtColor(inpainted_image_bgr, cv2.COLOR_BGR2RGB)

    # Convert the NumPy array back to PIL Image for consistent API usage
    inpainted_image_pil = Image.fromarray(inpainted_image_rgb)

    return inpainted_image_pil

In [13]:
model = load_model()

# Specify the target classes you want to remove by name
target_class_names = ["dog"]

# Convert class names to their corresponding IDs
target_classes = [class_name_to_id[name] for name in target_class_names]

inpainted_image = inpaint_image(model, 'dog6.jpg', target_classes)
inpainted_image.save('inpainted_image.jpg')

Using cache found in C:\Users\Aryan/.cache\torch\hub\pytorch_vision_v0.10.0


KeyboardInterrupt: 